In [1]:
import new_pywgraph as g

In [6]:
graph = g.Graph(
    nodes = {"A", "C", "B"}, 
    edges= {g.DirectedEdge("A", "B"), g.WeightedDirectedEdge("B", "C", 14), g.DirectedEdge("A", "C")}
)
graph.nodes

{'A', 'B', 'C'}

In [9]:
from typing import Any, Type
def check(a: Any, type_class: Type) -> bool: 
    return isinstance(a, type_class) or  issubclass(type(a), type_class)

In [11]:
type(3)

int

In [12]:
issubclass(int, float)

False